## Introduction to our first task: 'Dogs vs Cats'

In [0]:
# Safe for Colab
%matplotlib inline

The dataset is available at http://files.fast.ai/data/dogscats.zip. You can download it directly on your server by running the following line in your terminal. `wget http://files.fast.ai/data/dogscats.zip`. You should put the data in a subdirectory of this notebook's directory, called `data/`. Note that this data is already available in Crestle and the Paperspace fast.ai template.

In [0]:
# The dataset is available at http://files.fast.ai/data/dogscats.zip.
!mkdir -p data
!wget -q http://files.fast.ai/data/dogscats.zip
!unzip -q dogscats.zip -d data/

In [0]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [0]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

94658560/94653016 [==============================] - 15s 0us/step


In [0]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 347s 967ms/step - loss: 0.2238 - acc: 0.9469 - val_loss: 0.2192 - val_acc: 0.9496
Epoch 2/3
359/359 [==============================] - 340s 948ms/step - loss: 0.0887 - acc: 0.9689 - val_loss: 0.0566 - val_acc: 0.9864
Epoch 3/3
359/359 [==============================] - 340s 947ms/step - loss: 0.0685 - acc: 0.9757 - val_loss: 0.1777 - val_acc: 0.9632
CPU times: user 31min 17s, sys: 1min 12s, total: 32min 29s
Wall time: 17min 7s


In [0]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [11]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 366s 1s/step - loss: 0.0778 - acc: 0.9744 - val_loss: 7.6045 - val_acc: 0.5222
CPU times: user 11min 1s, sys: 35.4 s, total: 11min 36s
Wall time: 6min 7s
